REMEMBER: FIRST CREATE A COPY OF THIS FILE WITH A UNIQUE NAME AND DO YOUR WORK THERE. AND MAKE SURE YOU COMMIT YOUR CHANGES TO THE `hw3_submissions` BRANCH.

# Assignment 3 | Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=250>

In this assignment, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so you will need to account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into two pandas dataframes named `bus` and `ins`, respectively. Print the first 5 rows of each to inspect them.


In [1]:
import pandas as pd

In [2]:
bus = pd.read_csv('data/businesses.csv', encoding='ISO-8859-1')
ins = pd.read_csv('data/inspections.csv')

In [3]:
bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [4]:
ins.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [5]:
bus.shape

#There are 6315 records

(6315, 9)

#### Question 2b: How many unique business IDs are there?  

In [6]:
bus['business_id'].nunique()

#There are 6315 unique business IDs

6315

#### Question 2c: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [7]:
bus['name'].value_counts()

#Starbucks Coffee, Peet's Coffee & Tea, McDonalds, Walgreens, San Francisco Soup Company

STARBUCKS COFFEE              72
PEET'S COFFEE & TEA           24
MCDONALDS                     12
SAN FRANCISCO SOUP COMPANY    11
WALGREENS                     11
                              ..
D'MAIZE                        1
MISSION SHELL                  1
CASA GUADALUPE #2              1
MARINA DELI                    1
ANNAKOOT                       1
Name: name, Length: 5692, dtype: int64

## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values stored in python (i.e. data type)?

To answer this you might want to examine a particular entry.

In [8]:
bus.sample(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
5653,85299,PICCOLO FORNO,725 COLUMBUS AVE,San Francisco,CA,94133,NaN,NaN,NaN


In [9]:
bus.dtypes

#zipcode values are stored as an object because there are a mix of strings and numbers

business_id       int64
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
phone_number     object
dtype: object

#### Question 3b

What are the unique values of postal_code?

In [10]:
bus['postal_code'].unique()

array(['94104', '94109', '94133', '94110', '94122', '94103', '94115',
       '94131', '94111', '94117', '94107', '94108', '94102', '94132',
       '94105', '94134', '94124', '94116', '94121', '94118', '94112',
       '94123', '94114', '94127', '941102019', '941', '94545', '94066',
       '941033148', nan, '94158', '95105', '94013', '94130', 'CA',
       '92672', '94120', '94143', '94609', '94101', '00000', '94188',
       '94621', '94014', '94129', '94602', 'Ca'], dtype=object)

#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid 5-digit zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [11]:
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [12]:
bus_valid = bus[bus['postal_code'].isin(validZip)]
bus_valid['business_id'].nunique()

#There are 5999 businesses in this data frame

5999

## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are representative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude values, working with only the businesses that are in the list of valid zipcodes?

In [13]:
no_long = bus_valid[pd.isnull(bus_valid['longitude'])]

no_long['business_id'].nunique()
#2483 businesses are missing longitude values


2483

#### Question 4b

Create a new dataframe with one row for each valid zipcode. The dataframe should include the following three columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of businesses in that zipcode with missing `longitude` values.
3. `not_null_lon`: The number of businesses without missing `longitude` values.

In [14]:
yes_long = bus_valid[pd.notnull(bus_valid['longitude'])]

df1 = no_long[['business_id']].groupby(no_long['postal_code']).count().rename(columns={'business_id': 'null_lon'})

df2 = yes_long[['business_id']].groupby(yes_long['postal_code']).count().rename(columns={'business_id': 'not_null_lon'})

merged = pd.merge(df1, df2, on="postal_code", how="inner")

merged

,null_lon,not_null_lon
postal_code,,
94102,196,262
94103,252,306
94104,59,74
94105,100,126
94107,247,204
94108,82,142
94109,146,237
94110,251,334
94111,115,167


#### 4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [15]:
merged['missing_long']= merged['null_lon'] / (merged['null_lon'] + merged['not_null_lon']) * 100

merged.sort_values('missing_long', ascending=False).head()

,null_lon,not_null_lon,missing_long
postal_code,,,
94107,247,204,54.767184
94124,88,76,53.658537
94132,65,69,48.507463
94114,108,122,46.956522
94122,123,147,45.555556


# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [16]:
ins.shape

(15430, 4)

In [17]:
ins['business_id'].nunique()

#there are 5730 businesses, but 15430 inspection records, indicating on average there are just below 3 inspections/business

5730

#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe named `ins2` by copying `ins` and keeping only records with values of `type` that occur more than 10 times in the original table. In other words, eliminate records that have values of `type` that occur rarely (< 10 times). Check the result to make sure rare types are eliminated.

In [18]:
ins['type'].unique()
#ins takes on the values of 'routine' and compliant

array(['routine', 'complaint'], dtype=object)

In [19]:
ins[['business_id','type']].groupby('type').count()

,business_id
type,
complaint,1
routine,15429


In [20]:
ins2 = ins[(ins['type'] != 'complaint')]

ins2[['business_id','type']].groupby('type').count()

,business_id
type,
routine,15429


#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

Hint: we have seen multiple ways of doing this in class, includings `str` operations, `lambda` functions, `datetime` operations, and others. Choose the method that works best for you :)

In [21]:
ins.dtypes

business_id     int64
score           int64
date            int64
type           object
dtype: object

In [22]:
ins['date'] = ins['date'].astype(str)

In [23]:
ins['year'] = ins['date'].str[:4]
ins

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year? Try dropping records for any years with less than 50 inspections and store the result in a new dataframe named `ins3`.

In [24]:
ins['year'] = ins['year'].astype(int)
ins['year'].describe()
#ins covers 3 years, 2013 to 2016.

count    15430.000000
mean      2015.008749
std          0.867822
min       2013.000000
25%       2014.000000
50%       2015.000000
75%       2016.000000
max       2016.000000
Name: year, dtype: float64

In [25]:
keep = ins[['business_id']].groupby(ins['year']).count().rename(columns={'business_id': 'total'}) > 50
merge2 = pd.merge(ins, keep, on='year', how = 'left')
ins3 = merge2[(merge2['total'] == True)]
ins3['year'].describe()

count    15392.000000
mean      2015.013708
std          0.863127
min       2014.000000
25%       2014.000000
50%       2015.000000
75%       2016.000000
max       2016.000000
Name: year, dtype: float64

Let's examine only the inspections for one year: 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [26]:
ins3[(ins3['year'] == 2016)]

,business_id,score,date,type,year,total
0,10,82,20160503,routine,2016,True
3,19,94,20160513,routine,2016,True
6,24,98,20161005,routine,2016,True
7,24,96,20160311,routine,2016,True
10,31,84,20160816,routine,2016,True
...,...,...,...,...,...,...
15425,89515,98,20161103,routine,2016,True
15426,89701,100,20161206,routine,2016,True
15427,89790,96,20161129,routine,2016,True
15428,89803,100,20161215,routine,2016,True


### Question 6

#### Question 6a

Merge the business and 2016 inspections data, keeping all businesses regardless of whether they show up in the inspections file. Show the first several rows of the resulting dataframe.

In [32]:
ins4 = ins3[(ins3['year'] == 2016)]

everything = pd.merge(bus, ins4, on ='business_id', how = 'left')

everything.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,year,total
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044,82.0,20160503,routine,2016.0,True
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94.0,20160513,routine,2016.0,True
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,98.0,20161005,routine,2016.0,True
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,96.0,20160311,routine,2016.0,True
4,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,84.0,20160816,routine,2016.0,True


#### Question 6b
Print the 20 lowest rated businesses names, their addresses, and their ratings.

In [35]:
everything.sort_values('score').head(20)

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,year,total
4892,71373,GOLDEN RIVER RESTAURANT,5827 GEARY BLVD,San Francisco,CA,94121,37.780144,-122.482040,+14156685105,52.0,20161031,routine,2016.0,True
7325,87652,POKI TIME,2101 LOMBARD ST,San Francisco,CA,94123,NaN,NaN,+14156366868,55.0,20160721,routine,2016.0,True
6281,81667,CRAZY PEPPER,2257 SAN JOSE AVE,San Francisco,CA,94112,NaN,NaN,+14157348687,55.0,20160901,routine,2016.0,True
4873,71232,UNCLE CAFE,65 WAVERLY PL,San Francisco,CA,94108,37.794034,-122.406794,+14155701380,55.0,20160308,routine,2016.0,True
3008,34585,HING WANG BAKERY,339 JUDAH ST,San Francisco,CA,94122,37.762174,-122.465809,+14156813928,56.0,20160201,routine,2016.0,True
5627,78328,GOLDEN WOK,295 B ORIZABA AVE,San Francisco,CA,94132,NaN,NaN,+14155868000,56.0,20160824,routine,2016.0,True
5383,76615,TOP'S CAFE,2460 SAN BRUNO AVE,San Francisco,CA,94134,NaN,NaN,+14157156868,57.0,20160504,routine,2016.0,True
6067,80591,L & G VIETNAMESE SANDWICH,602 EDDY ST,San Francisco,CA,94109,NaN,NaN,+14158856402,57.0,20160129,routine,2016.0,True
836,2716,RED A BAKERY,634 CLEMENT ST,San Francisco,CA,94118,37.782888,-122.466101,+14155663888,57.0,20160323,routine,2016.0,True
4209,67330,SAN TUNG RESTAURANT LLC,1031 IRVING ST,San Francisco,CA,94122,37.763891,-122.468805,+14152420828,57.0,20160825,routine,2016.0,True


## Done!

Now commit this notebook to your `hw3_submissions` branch, push it to your GitHub repo, and open a PR!